In [2]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
from datetime import datetime

import requests
import json

In [5]:
uri = 'https://api.football-data.org/v4/teams/67/matches?status=FINISHED'
headers = { 'X-Auth-Token': '41febec7ab9f4556a48c9df439e1fe9c' }

response = requests.get(uri, headers=headers)
for nclFinished in response.json()['matches']:
  print(json.dumps(nclFinished, indent=2))

{
  "area": {
    "id": 2072,
    "name": "England",
    "code": "ENG",
    "flag": "https://crests.football-data.org/770.svg"
  },
  "competition": {
    "id": 2021,
    "name": "Premier League",
    "code": "PL",
    "type": "LEAGUE",
    "emblem": "https://crests.football-data.org/PL.png"
  },
  "season": {
    "id": 733,
    "startDate": "2021-08-13",
    "endDate": "2022-05-22",
    "currentMatchday": 38,
    "winner": null
  },
  "id": 327356,
  "utcDate": "2021-08-15T13:00:00Z",
  "status": "FINISHED",
  "matchday": 1,
  "stage": "REGULAR_SEASON",
  "group": null,
  "lastUpdated": "2022-06-16T08:20:23Z",
  "homeTeam": {
    "id": 67,
    "name": "Newcastle United FC",
    "shortName": "Newcastle",
    "tla": "NEW",
    "crest": "https://crests.football-data.org/67.png"
  },
  "awayTeam": {
    "id": 563,
    "name": "West Ham United FC",
    "shortName": "West Ham",
    "tla": "WHU",
    "crest": "https://crests.football-data.org/563.png"
  },
  "score": {
    "winner": "AWAY_TE

In [43]:
df2 = pd.json_normalize(nclFinished).T
df2.head(30)

,0
id,326989
utcDate,2022-05-22T15:00:00Z
status,FINISHED
matchday,38
stage,REGULAR_SEASON
group,None
lastUpdated,2022-06-16T08:20:23Z
referees,"[{'id': 11585, 'name': 'Craig Pawson', 'type':..."
area.id,2072
area.name,England


In [ ]:
uri = 'https://api.football-data.org/v4/competitions/PL/matches?status=SCHEDULED&limit=2'
headers = { 'X-Auth-Token': '41febec7ab9f4556a48c9df439e1fe9c' }

response = requests.get(uri, headers=headers)
for match in response.json()['matches']:
  match


In [32]:

df = pd.read_json(json.dumps(nclmatch), lines=True, orient='columns')
df.head(10)

,area,competition,season,id,utcDate,status,matchday,stage,group,lastUpdated,homeTeam,awayTeam,score,odds,referees
0,"{'id': 2072, 'name': 'England', 'code': 'ENG',...","{'id': 2021, 'name': 'Premier League', 'code':...","{'id': 1490, 'startDate': '2022-08-05', 'endDa...",416011,2023-05-28T15:00:00Z,TIMED,38,REGULAR_SEASON,NaN,2022-06-16T15:49:17Z,"{'id': 61, 'name': 'Chelsea FC', 'shortName': ...","{'id': 67, 'name': 'Newcastle United FC', 'sho...","{'winner': None, 'duration': 'REGULAR', 'fullT...",{'msg': 'Activate Odds-Package in User-Panel t...,[]


In [37]:

df = pd.json_normalize(nclmatch)
df.head()

,id,utcDate,status,matchday,stage,group,lastUpdated,referees,area.id,area.name,...,awayTeam.shortName,awayTeam.tla,awayTeam.crest,score.winner,score.duration,score.fullTime.home,score.fullTime.away,score.halfTime.home,score.halfTime.away,odds.msg
0,416011,2023-05-28T15:00:00Z,TIMED,38,REGULAR_SEASON,None,2022-06-16T15:49:17Z,[],2072,England,...,Newcastle,NEW,https://crests.football-data.org/67.png,None,REGULAR,None,None,None,None,Activate Odds-Package in User-Panel to retriev...
